In [130]:
# Iván Andrés Trujillo Abella
# ivantrujillo1229@gmail.com

In [131]:
!pip install tableone

Defaulting to user installation because normal site-packages is not writeable


In [132]:
import re
import pandas as pd
import numpy as np
from tableone import TableOne
from scipy import stats

In [133]:
url  = "https://raw.githubusercontent.com/it-ces/Datasets/main/type_of_credit.csv"

In [134]:
df = pd.read_csv(url)

In [135]:
df['Default'].value_counts(normalize=True)

Default
0    0.899
1    0.101
Name: proportion, dtype: float64

In [136]:
df['Monthly-payment'].describe()

count    1.000000e+03
mean     2.700276e+05
std      5.187051e+05
min      2.454283e+02
25%      7.029921e+03
50%      8.658389e+04
75%      3.259568e+05
max      5.353655e+06
Name: Monthly-payment, dtype: float64

In [137]:
df['type_of_credit'].unique()

array(['Avance', 'Microempresario ', 'Micro Convenio ',
       'Crédito  productivo urbano Micro ', ' Libre Inversion',
       'Independientes Tipo A Apalancamiento  ',
       'Libre Inversion Empleados ',
       'Micro Empresarial Activos Fijos do Credito',
       'Microcredito Empresarial Activos Fijos', 'Premium Pensionados',
       'Credito Digital', 'Microcredito Cupo de Crédito',
       'Credito Preaprobado Consumo', '     Readquisición Empleados',
       'Premium', 'Pensionados  Apalancamiento  ',
       ' volante Independientes', '  Pensionados ',
       'Trabajadores Prosperando', 'productivo urbano ',
       'Crédito productivo urbano DIGITAL',
       'Libre Inversion Pensionados ', 'Micro ',
       'Micro Empresar Activos Fijos er Credito',
       'Independientes Tipo B Apalancamiento  ', ' Pensionados',
       '  Empleados ', '  Empleados Sustitución Pasivos',
       ' volante Pensionados', 'Preferencial',
       ' volante Independientes Micro', ' Independientes',
       ' 

In [138]:
df['type_of_credit']= df['type_of_credit'].apply(lambda x : str(x).upper().replace(" ",""))

In [139]:
df['type_of_credit'].unique()

array(['AVANCE', 'MICROEMPRESARIO', 'MICROCONVENIO',
       'CRÉDITOPRODUCTIVOURBANOMICRO', 'LIBREINVERSION',
       'INDEPENDIENTESTIPOAAPALANCAMIENTO', 'LIBREINVERSIONEMPLEADOS',
       'MICROEMPRESARIALACTIVOSFIJOSDOCREDITO',
       'MICROCREDITOEMPRESARIALACTIVOSFIJOS', 'PREMIUMPENSIONADOS',
       'CREDITODIGITAL', 'MICROCREDITOCUPODECRÉDITO',
       'CREDITOPREAPROBADOCONSUMO', 'READQUISICIÓNEMPLEADOS', 'PREMIUM',
       'PENSIONADOSAPALANCAMIENTO', 'VOLANTEINDEPENDIENTES',
       'PENSIONADOS', 'TRABAJADORESPROSPERANDO', 'PRODUCTIVOURBANO',
       'CRÉDITOPRODUCTIVOURBANODIGITAL', 'LIBREINVERSIONPENSIONADOS',
       'MICRO', 'MICROEMPRESARACTIVOSFIJOSERCREDITO',
       'INDEPENDIENTESTIPOBAPALANCAMIENTO', 'EMPLEADOS',
       'EMPLEADOSSUSTITUCIÓNPASIVOS', 'VOLANTEPENSIONADOS',
       'PREFERENCIAL', 'VOLANTEINDEPENDIENTESMICRO', 'INDEPENDIENTES',
       'READQUISICIÓNMICROERCRÉDITO', 'EMPLEADOSDCTONOMINAA',
       'VOLANTEMICROEMPRESARIOSNUEVOS', 'EMPLEADOSDEPERSONASNATURALES',


In [140]:
def regNewVar(target, col_name, regex, df):
    df[col_name]  = 0
    for index in df.index:
        if bool(re.search(regex, df.loc[index,target], re.IGNORECASE)):
            df.loc[index, col_name] = 1
    return  df[col_name]

In [141]:
map_regex= {'Crediavance':r'(credi)?avance',
           'Microcredito':r'^(?!.*(independientes|productivo|empleados)).*(micro)',
           'Productivo':r'produ(c)?.*',
           'Libre Inversion' : r'^(?!.*(pensionados|independientes|empleados)).*(libreinversion)',
           'Pensionados' : r'pensionado(s)?',
           'Empleados': r'empleado(s)?',
           'Independientes': r'independiente(s)?',
           #'Agropecuario' : r'Agropecuario',
            'Digital' : r'^(?!.*(productivo|microcr[eé]dito|empleados)).*(digital)'}

In [142]:
df.tail()

,Monthly-payment,type_of_credit,Default
995,11206.020537,AVANCE,0
996,76542.677719,PREMIUM,0
997,16632.268859,AVANCE,0
998,3014.069485,AVANCE,0
999,469585.218164,CREDITODIGITAL,0


In [143]:
regNewVar('type_of_credit', 'Digital', map_regex['Digital'], df)

0      0
1      0
2      0
3      0
4      0
      ..
995    0
996    0
997    0
998    0
999    1
Name: Digital, Length: 1000, dtype: int64

In [144]:
for reg in map_regex:
    df[reg] = regNewVar('type_of_credit', reg, map_regex[reg], df)

In [145]:
df['otro tipo de crédito'] = (df[list(map_regex.keys())].sum(axis=1)==0).transform(lambda x: int(x))

In [146]:
df

,Monthly-payment,type_of_credit,Default,Digital,Crediavance,Microcredito,Productivo,Libre Inversion,Pensionados,Empleados,Independientes,otro tipo de crédito
0,4253.073763,AVANCE,0,0,1,0,0,0,0,0,0,0
1,360815.648703,MICROEMPRESARIO,1,0,0,1,0,0,0,0,0,0
2,299912.171531,MICROCONVENIO,0,0,0,1,0,0,0,0,0,0
3,333935.745843,CRÉDITOPRODUCTIVOURBANOMICRO,0,0,0,0,1,0,0,0,0,0
4,4207.902271,AVANCE,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
995,11206.020537,AVANCE,0,0,1,0,0,0,0,0,0,0
996,76542.677719,PREMIUM,0,0,0,0,0,0,0,0,0,1
997,16632.268859,AVANCE,0,0,1,0,0,0,0,0,0,0
998,3014.069485,AVANCE,0,0,1,0,0,0,0,0,0,0


In [147]:
df[df[list(map_regex.keys())].sum(axis=1)>1]['type_of_credit'].unique()

array([], dtype=object)

In [148]:
df[list(map_regex.keys())].sum()

Crediavance        471
Microcredito       151
Productivo          11
Libre Inversion     80
Pensionados         98
Empleados           59
Independientes      39
Digital             52
dtype: int64

In [149]:
df[df[list(map_regex.keys())].sum(axis=1)>=1] # Fix this!!

,Monthly-payment,type_of_credit,Default,Digital,Crediavance,Microcredito,Productivo,Libre Inversion,Pensionados,Empleados,Independientes,otro tipo de crédito
0,4253.073763,AVANCE,0,0,1,0,0,0,0,0,0,0
1,360815.648703,MICROEMPRESARIO,1,0,0,1,0,0,0,0,0,0
2,299912.171531,MICROCONVENIO,0,0,0,1,0,0,0,0,0,0
3,333935.745843,CRÉDITOPRODUCTIVOURBANOMICRO,0,0,0,0,1,0,0,0,0,0
4,4207.902271,AVANCE,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
994,16753.908676,AVANCE,0,0,1,0,0,0,0,0,0,0
995,11206.020537,AVANCE,0,0,1,0,0,0,0,0,0,0
997,16632.268859,AVANCE,0,0,1,0,0,0,0,0,0,0
998,3014.069485,AVANCE,0,0,1,0,0,0,0,0,0,0


In [150]:
df['Another'] = (df[list(map_regex.keys())].sum(axis=1)==0).transform(lambda x: int(x))

In [151]:
df

,Monthly-payment,type_of_credit,Default,Digital,Crediavance,Microcredito,Productivo,Libre Inversion,Pensionados,Empleados,Independientes,otro tipo de crédito,Another
0,4253.073763,AVANCE,0,0,1,0,0,0,0,0,0,0,0
1,360815.648703,MICROEMPRESARIO,1,0,0,1,0,0,0,0,0,0,0
2,299912.171531,MICROCONVENIO,0,0,0,1,0,0,0,0,0,0,0
3,333935.745843,CRÉDITOPRODUCTIVOURBANOMICRO,0,0,0,0,1,0,0,0,0,0,0
4,4207.902271,AVANCE,0,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,11206.020537,AVANCE,0,0,1,0,0,0,0,0,0,0,0
996,76542.677719,PREMIUM,0,0,0,0,0,0,0,0,0,1,1
997,16632.268859,AVANCE,0,0,1,0,0,0,0,0,0,0,0
998,3014.069485,AVANCE,0,0,1,0,0,0,0,0,0,0,0


In [152]:
df[df[list(map_regex.keys()) + ['Another']].sum(axis=1)>1]

,Monthly-payment,type_of_credit,Default,Digital,Crediavance,Microcredito,Productivo,Libre Inversion,Pensionados,Empleados,Independientes,otro tipo de crédito,Another


In [153]:
cats = list(map_regex.keys()) + ['otro tipo de crédito']
df['type_of_credit_fixed'] = ''
for index in df.index:
    for cat in cats:
        if df.loc[index,cat]==1:
            df.loc[index,'type_of_credit_fixed'] = cat

In [163]:
df['type_of_credit_fixed'].value_counts()

type_of_credit_fixed
Crediavance             471
Microcredito            151
Pensionados              98
Libre Inversion          80
Empleados                59
Digital                  52
Independientes           39
otro tipo de crédito     39
Productivo               11
Name: count, dtype: int64

In [155]:
df = df[['Monthly-payment','type_of_credit_fixed', 'Default']]

In [156]:
df

,Monthly-payment,type_of_credit_fixed,Default
0,4253.073763,Crediavance,0
1,360815.648703,Microcredito,1
2,299912.171531,Microcredito,0
3,333935.745843,Productivo,0
4,4207.902271,Crediavance,0
...,...,...,...
995,11206.020537,Crediavance,0
996,76542.677719,otro tipo de crédito,0
997,16632.268859,Crediavance,0
998,3014.069485,Crediavance,0


In [157]:
def is_binary(df_, nums):
    df = df_.copy()
    variables = []
    for var in nums:
        flag = True
        unique = df_[var].unique()
        for value in unique:
            if value not in [0, 1, np.nan, 0.0, 1.0]:
                flag = False
        if flag == True:
            variables.append(var)
    return variables


def breakdown_vars(df):
    """
    This function allow us categorize accodign to numerical or not
    """
    categorial = []
    nonormal = []
    normal = []
    binaries = is_binary(df, df.columns)
    for t in df.columns:
            if (df[t].dtypes.name=="object" or df[t].dtypes.name=='category') and t not in binaries:
                categorial.append(t)
            if (df[t].dtypes=="int64" or df[t].dtypes=="float64") and t not in binaries:
              n,p = stats.shapiro(df[t])
              if p<0.05:
                nonormal.append(t)
              else:
                normal.append(t)
    return categorial, binaries, nonormal, normal

In [158]:
df['Default'] = [str(value) for value in df['Default']]

/tmp/ipykernel_466/3966258823.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Default'] = [str(value) for value in df['Default']]


In [159]:
categorial, binaries, nonormal, normal = breakdown_vars(df)

In [160]:
mytable = TableOne(df,
                   categorical=categorial,
                   nonnormal=nonormal,
                   groupby= 'type_of_credit_fixed',
                   pval=True)

/home/vscode/.local/lib/python3.10/site-packages/tableone/tableone.py:322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[self._categorical] = handle_categorical_nulls(data[self._categorical])


In [161]:
mytable

Grouped by type_of_credit_fixed                                                                                                                                                                                                                                                                                                              
                                                                             Missing                    Overall              Crediavance                       Digital                     Empleados                Independientes                Libre Inversion                  Microcredito                   Pensionados                    Productivo           otro tipo de crédito P-Value
n                                                                                                          1000                      471                            52                            59                            39                             80                           151                            98                            11                             39        
Monthly-payment, median [Q1,Q3]                                                    0  86583.9 [7029.9,325956.8]  6915.0 [2840.6,13925.2]  267952.2 [150717.2,375507.2]  220988.3 [169271.5,298596.3]  458503.5 [285202.6,729596.5]  647646.5 [348417.9,1139685.2]  242230.4 [138726.9,376833.2]  394315.3 [228712.0,666965.5]  469048.8 [302299.6,595854.9]  251299.3 [104382.7,1025200.6]  <0.001
type_of_credit_fixed, n (%)     Crediavance                                                          471 (47.1)              471 (100.0)                                                                                                                                                                                                                                                    <0.001
                                Digital                                                                52 (5.2)                                             52 (100.0)                                                                                                                                                                                                                            
                                Empleados                                                              59 (5.9)                                                                           59 (100.0)                                                                                                                                                                                              
                                Independientes                                                         39 (3.9)                                                                                                         39 (100.0)                                                                                                                                                                
                                Libre Inversion                                                        80 (8.0)                                                                                                                                        80 (100.0)                                                                                                                                 
                                Microcredito                                                         151 (15.1)                                                                                                                                                                     151 (100.0)                                                                                                   
                                Pensionados                                                            98 (9.8)                                                                        